In [7]:
#from mpmath import *
#mp.dps = 15; mp.pretty = True
from scipy.integrate import dblquad
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
import sys,os
#from stylelib.ase1_styles import ase1_sims_stl
#plt.style.use(ase1_sims_stl)

In [2]:
#a,b = 1., -100.
# Integral over gaussian function
#y = quad(lambda x: exp(-a*(x-b)**2), linspace(-1100, 1000,100))
#print y*y

In [3]:
# Integral over gaussian derivatives of known values, TEST
#a,b = 1000., 36. # High 'a' turns integral into a delta function
#y = quad(lambda x: x*exp(-a*(x-b)**2), linspace(-200, 100, 100))
#print y*sqrt(a/pi)
# Integrals seem to always converge when the number of intervals ~= the coefficient of the exponent

In [4]:
# Double integrals of known values, TEST
#k = 1000.0 # spring constant
#f1 = lambda x1, x2: exp(-0.5*k*(x1**2 + x2**2 - 2*sqrt(1-(1/k))*x1*x2))
#q = quad(f1, linspace(-10, 10, 12), linspace(-10, 10, 12))
#q = quad(f1, [-10, 10], [-10, 10])
#print (q*sqrt(k))/(2.*pi)

In [5]:
# How does scipy's double quad method stack up to sympy? TEST
f1 = lambda x1, x2: np.exp(-0.5*k*(np.power(x1,2) + np.power(x2,2) - 2.*np.sqrt(1-(1/k))*x1*x2))
q, _= dblquad(f1, -10,10, lambda x2:-10, lambda x2:10, epsabs=0, epsrel=1.e-8)
#q = quad(f1, [-10, 10], [-10, 10])
print (q*sqrt(k))/(2.*np.pi)

NameError: name 'k' is not defined

In [ ]:
# Force between parallel filaments of equal length, TESTED
k = 4.56 # spring constant
b = 1. # beta
yo = 1. # Horizontal separation
Dr = 10. # COM separation
ho = 0. # equilibrium length
c = 100. # Crosslinker affinity * fugacity
hL = 10. # Length of filaments
#fdr = lambda x1, x2, r:-1.*c*k*(x1 - x2 + r)*(1. - (ho/np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))))*np.exp(-.5*k*b*np.power(np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))-ho, 2))
fdr = lambda x1, x2, r:-1.*c*k*(x1 - x2 + r)*np.exp(-.5*k*b*np.power(np.power(x1-x2+r,2)+np.power(yo,2), 2))
#print fdr(0,0)
f, err = dblquad(fdr, -hL, hL, lambda x2:-hL, lambda x2:hL, args=[0.], epsabs=0, epsrel=1.e-13)
print f, err

In [ ]:
# Scan over multiple values of Delta r
Dr_list = np.linspace(-22, 22, 100).tolist()
f_list = [dblquad(fdr, -hL, hL, lambda x2:-hL, lambda x2:hL, args = [r], epsabs=0, epsrel=1.e-13) for r in Dr_list]
f_arr = np.array(f_list)

In [ ]:
# Graph scan over area
fig, ax = plt.subplots(figsize=(10,7))
ax.errorbar(np.array(Dr_list)*25., f_arr[:,0]*.1644*16, yerr=f_arr[:,1]*.1644)
#ax.set_xlim((-22,22))
ax.set_xlabel(r'Separation of MT centers $\Delta r$ (nm)')
ax.set_ylabel(r'Total crosslinker force $F_{\rm cl}$ (pN)')
plt.show()


In [ ]:
# Partition function for parallel filaments as a function of delta r
Ndr = lambda x1, x2, r:c*np.exp(-.5*k*b*np.power(np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))-ho, 2))

In [ ]:
N_list = [dblquad(Ndr, -hL, hL, lambda x2:-hL, lambda x2:hL, args = [r], epsabs=0, epsrel=1.e-13) for r in Dr_list]
N_arr = np.array(N_list)

In [ ]:
# Graph number of crosslinkers based off partition function
fig, ax = plt.subplots(figsize=(10,7))
ax.errorbar(np.array(Dr_list)*25., N_arr[:,0], yerr=N_arr[:,1])
#ax.set_xlim((-22,22))
ax.set_xlabel(r'Separation of MT centers $\Delta r$ (nm)')
ax.set_ylabel(r'Total number of crosslinkers $N_{\rm cl}$')
plt.show()

In [ ]:
# Canonical force calculation
fig, ax = plt.subplots(figsize=(10,7))

ax.set_xlim((40,500))
ax.set_ylim((-4, 0))
ax.plot(np.array(Dr_list)*25., np.divide(f_arr[:,0],N_arr[:,0]))
ax.set_xlabel(r'Separation of MT COMs $\Delta r$ (nm)')
ax.set_ylabel(r'Total force from crosslinkers $F_{\rm cl}$ (pN)')
plt.show()

In [ ]:
# Crosslinkers with some equilibrium length
ho = 2.28
k = 31.25
Dr_list = np.linspace(-22, 22, 100).tolist()
fho_list = [dblquad(fdr, -hL, hL, lambda x2:-hL, lambda x2:hL, args = [r], epsabs=0, epsrel=1.e-13) for r in Dr_list]
fho_arr = np.array(f_list)

In [ ]:
# Graph scan over area
fig, ax = plt.subplots(figsize=(10,7))
ax.errorbar(np.array(Dr_list)*25., fho_arr[:,0]*.1644, yerr=fho_arr[:,1]*.1644)
ax.errorbar(np.array(Dr_list)*25., f_arr[:,0]*.1644, yerr=f_arr[:,1]*.1644, c='r')
#ax.set_xlim((-22,22))
ax.set_xlabel(r'Separation of MT COMs $\Delta r$ (nm)')
ax.set_ylabel(r'Total crosslinker force $F_{\rm cl}$ (pN)')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
Fs = 6.08 # Crosslinker stall force
maxx = np.sqrt(np.power(Fs/k,2)-np.power(yo,2))
ax.errorbar(np.array(Dr_list)*25., -k*maxx*N_arr[:,0]*.1644, yerr=N_arr[:,1]*.1644)
#ax.set_xlim((-22,22))
ax.set_xlabel(r'Separation of MT centers $\Delta r$ (nm)')
ax.set_ylabel(r'Total motor force $F_{\rm cl}$ (pN)')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
Fs = 6.08 # Crosslinker stall force
maxx = np.sqrt(np.power(Fs/k,2)-np.power(yo,2))
ax.errorbar(N_arr[:,0], -k*maxx*N_arr[:,0]*.1644, yerr=N_arr[:,1]*.1644)
#ax.set_xlim((-22,22))
ax.set_xlabel(r'Average number of motors')
ax.set_ylabel(r'Total motor force $F_{\rm cl}$ (pN)')
plt.show()

In [8]:
param_path = Path('/Users/adamlamson/projects/DATA/XlinkProb/testing/FP_GenOrient_StaticXlinks_tests/19-06-21_GO_para_test/FP_param.yaml')

In [13]:
with open(param_path,'r') as fp:
    params = yaml.safe_load(fp)
# Number of crosslinks between filaments of equal length
k = params['ks']
b = params['beta']
dR = np.asarray(params['R2_pos']) - np.asarray(params['R1_pos'])
y = dR[2]
x = dR[1] # COM separation
ho = params['ho'] # equilibrium length
co = params['co'] # Crosslinker affinity * fugacity
hL1 = params['L1']*.5 # Length of filaments
hL2 = params['L2']*.5
#fdr = lambda x1, x2, r:-1.*c*k*(x1 - x2 + r)*(1. - (ho/np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))))*np.exp(-.5*k*b*np.power(np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))-ho, 2))
dN = lambda x1, x2, r: co*np.exp(-.5*k*b*np.power(np.sqrt(np.power(x1-x2+x,2)+np.power(y,2))-ho, 2))
#print fdr(0,0)
N, err = dblquad(dN, -hL1, hL1, lambda x2:-hL2, lambda x2:hL2, args=[0.], epsabs=0, epsrel=1.e-13)
print(N, err)

227.13144227534823 2.6104023252453527e-12


In [14]:
param_path = Path('/Users/adamlamson/projects/DATA/XlinkProb/testing/FP_GenOrient_StaticXlinks_tests/19-06-21_GO_para_test/FP_param.yaml')
with open(param_path,'r') as fp:
    params = yaml.safe_load(fp)
k = params['ks']
b = params['beta']
dR = np.asarray(params['R2_pos']) - np.asarray(params['R1_pos'])
r = np.linalg.norm(dR)
rhat = dR/r
u1 = np.asarray(params['R1_vec'])
u1 /= np.linalg.norm(u1)
u2 = np.asarray(params['R2_vec'])
u2 /= np.linalg.norm(u2)
ho = params['ho'] # equilibrium length
co = params['co'] # Crosslinker affinity * fugacity
hL1 = params['L1']*.5 # Length of filaments
hL2 = params['L2']*.5
#fdr = lambda x1, x2, r:-1.*c*k*(x1 - x2 + r)*(1. - (ho/np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))))*np.exp(-.5*k*b*np.power(np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))-ho, 2))
dN = lambda x1, x2, r: co*np.exp(-.5*k*b*np.power(np.linalg.norm(dR + (x2*u2)-(x1*u1))-ho, 2))
#print fdr(0,0)
N, err = dblquad(dN, -hL1, hL1, lambda x2:-hL2, lambda x2:hL2, args=[0.], epsabs=0, epsrel=1.e-13)
print(N, err)

227.13144227534823 2.6104023252453527e-12


In [15]:
param_path = Path('/Users/adamlamson/projects/DATA/XlinkProb/testing/FP_GenOrient_StaticXlinks_tests/19-06-21_GO_perp_test/FP_param.yaml')
with open(param_path,'r') as fp:
    params = yaml.safe_load(fp)
k = params['ks']
b = params['beta']
dR = np.asarray(params['R2_pos']) - np.asarray(params['R1_pos'])
r = np.linalg.norm(dR)
rhat = dR/r
u1 = np.asarray(params['R1_vec'])
u1 /= np.linalg.norm(u1)
u2 = np.asarray(params['R2_vec'])
u2 /= np.linalg.norm(u2)
ho = params['ho'] # equilibrium length
co = params['co'] # Crosslinker affinity * fugacity
hL1 = params['L1']*.5 # Length of filaments
hL2 = params['L2']*.5
#fdr = lambda x1, x2, r:-1.*c*k*(x1 - x2 + r)*(1. - (ho/np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))))*np.exp(-.5*k*b*np.power(np.sqrt(np.power(x1-x2+r,2)+np.power(yo,2))-ho, 2))
dN = lambda x1, x2, r: co*np.exp(-.5*k*b*np.power(np.linalg.norm(dR + (x2*u2)-(x1*u1))-ho, 2))
#print fdr(0,0)
N, err = dblquad(dN, -hL1, hL1, lambda x2:-hL2, lambda x2:hL2, args=[0.], epsabs=0, epsrel=1.e-13)
print(N, err)

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


29.527908538569765 4.206487450666059e-13
